In [1]:
import os
import sys
import json
import time
import requests
import urllib.request
from datetime import datetime
from requests.auth import HTTPBasicAuth

In [3]:
# output folder name
# wet months
# scene_folder_name = '201701'
# scene_folder_name = '201702'

# dry months
scene_folder_name = '201709'
# scene_folder_name = '201708'
# scene_folder_name = '201707'

# set up session
item_type = "PSScene"
item_types = ["PSScene"]
URL = "https://api.planet.com/data/v1"
os.environ['PL_API_KEY'] = 'xxxx' # enter api key here
PLANET_API_KEY = os.getenv('PL_API_KEY')
session = requests.Session()
session.auth = (PLANET_API_KEY, "")
res = session.get(URL)
quick_url = "{}/quick-search".format(URL) # setup the quick search endpoint url

print("downloading scenes for: " + scene_folder_name)

# setup folder structure
# data_folder = r'data/scenes/' + scene_folder_name
data_folder = r'D:/Users/jpatel27/ponds_wlp/data/scenes/' + scene_folder_name

if not os.path.exists(data_folder):
    os.makedirs(data_folder)  # create folder if it does not exist

with open('D:/Users/jpatel27/ponds_wlp/data/calakmul.geojson', 'r') as j:
    os.chdir(data_folder)
    # Read AOI as geojson
    geojson_geometry = json.loads(j.read())

    # ---filters---
    # 1. get images that overlap with our AOI
    geometry_filter = {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": geojson_geometry
    }
    # 2. get images acquired within a date range

    # january
    # date_range_filter = {
    #   "type": "DateRangeFilter",
    #   "field_name": "acquired",
    #   "config": {
    #     "gte": "2017-01-01T00:00:00.000Z",
    #     "lte": "2017-01-31T00:00:00.000Z"
    #   }
    # }

    # september
    date_range_filter = {
      "type": "DateRangeFilter",
      "field_name": "acquired",
      "config": {
        "gte": "2017-09-01T00:00:00.000Z",
        "lte": "2017-09-30T00:00:00.000Z"
      }
    }
    # 3. only get images which have <50% cloud coverage
    cloud_cover_filter = {
      "type": "RangeFilter",
      "field_name": "cloud_cover",
      "config": {
        "lte": 0.5
      }
    }
    # combine our geo, date, cloud filters
    combined_filter = {
      "type": "AndFilter",
      "config": [geometry_filter, date_range_filter, cloud_cover_filter]
    }

    # setup the request
    request = {
        "item_types": item_types,
        "filter": combined_filter
    }

    # Send the POST request to the API quick search endpoint
    res = session.post(quick_url, json=request)

    # Assign the response to a variable
    geojson = res.json()
    # print(geojson)
    # print(geojson.keys())

    # extract image IDs only
    image_ids = [feature['id'] for feature in geojson['features']]
    print(image_ids)

    # all available assets for image id
#     for i in range(len(image_ids)):
    for i in range(223,len(image_ids)):
        print("current scene:" , image_ids[i])
        id0 = image_ids[i]
        id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)

        # Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
        result = \
            requests.get(
                id0_url,
                auth=HTTPBasicAuth(PLANET_API_KEY, '')
            )

        # List of asset types available for this particular satellite image
        result_keys = (result.json().keys())
        # print(result_keys)

        if "ortho_analytic_4b_sr" in result.json():
            # check asset status
            status = result.json()['ortho_analytic_4b_sr']['status']

            # activate asset
            # Parse out useful links
            # insert product here
            links = result.json()[u"ortho_analytic_4b_sr"]["_links"]
            self_link = links["_self"]
            activation_link = links["activate"]

            # Request activation of the 'analytic' asset:
            activate_result = \
                requests.get(
                    activation_link,
                    auth=HTTPBasicAuth(PLANET_API_KEY, '')
                )
            # assign inital status to cur_activation_status
            cur_activation_status = status
            print("current status ", datetime.now(), " ", status)
            # if status is already active
            # comment out if you only need inactive scenes
            if status == 'active':
                # retrieve download link
                activation_status_result = \
                    requests.get(
                        self_link,
                        auth=HTTPBasicAuth(PLANET_API_KEY, '')
                    )
                download_link = activation_status_result.json()["location"]
                # print(download_link)
                urllib.request.urlretrieve(download_link, image_ids[i])
                print("download complete")
                sys.stdout.flush()

            # if asset is not yet active
            if status != 'active':
                while cur_activation_status != 'active':
                    # poll every 30 seconds
                    time.sleep(30)
                    activation_status_result = \
                        requests.get(
                            self_link,
                            auth=HTTPBasicAuth(PLANET_API_KEY, '')
                        )
                    # update cur_activation_status to current status
                    cur_activation_status = activation_status_result.json()["status"]
                    print("current status ", datetime.now(), " ", cur_activation_status)
                    # if asset is activated
                    if cur_activation_status == 'active':
                        # retrieve download link
                        download_link = activation_status_result.json()["location"]
                        urllib.request.urlretrieve(download_link, image_ids[i])
                        print("download complete: ", download_link)
                        continue
        else:
            print(image_ids[i], ' n/a')

downloading scenes for: 201709
['20170924_214635_1_0c24', '20170924_225738_0c38', '20170922_223240_0c24', '20170924_225740_0c38', '20170924_225739_0c38', '20170922_223242_0c24', '20170922_223242_1_0c24', '20170924_225740_1_0c38', '20170922_223241_0c24', '20170922_223240_1_0c24', '20170902_154857_0f52', '20170902_154929_0f34', '20170902_154939_0f34', '20170902_154934_0f34', '20170902_154933_0f34', '20170902_154932_0f34', '20170902_154931_0f34', '20170902_154930_0f34', '20170929_194426_0d06', '20170929_194425_0d06', '20170927_170654_0f53', '20170926_154643_1035', '20170926_154626_1035', '20170926_154642_1035', '20170926_154640_1035', '20170926_154639_1035', '20170926_154644_1035', '20170926_154622_1035', '20170926_154939_1008', '20170926_154932_1008', '20170926_154931_1008', '20170926_154938_1008', '20170926_154937_1008', '20170926_154935_1008', '20170925_170550_1050', '20170925_170548_1050', '20170925_170534_1050', '20170925_170547_1050', '20170925_170545_1050', '20170925_170544_1050', 

download complete
current scene: 20170922_154749_1021
current status  2022-04-06 16:46:51.217166   active
download complete
current scene: 20170922_154743_1021
current status  2022-04-06 16:46:54.555116   active
download complete
current scene: 20170922_154742_1021
current status  2022-04-06 16:46:57.971344   active
download complete
current scene: 20170922_154802_1021
current status  2022-04-06 16:47:02.428699   active
download complete
current scene: 20170922_154752_1021
current status  2022-04-06 16:47:09.587417   active
download complete
current scene: 20170922_154748_1021
current status  2022-04-06 16:47:13.362057   active
download complete
